In [ ]:
import pandas as pd
import os
import tensorflow as tf

from nightingale.model.classifier_head import ClassifierHead
from nightingale.model.yamnet_base import YamnetEmbedding



### Load and Explore birdclef-2024 data (pre conversion)

In [16]:
# Read train meta data
train_metadata_path = "../data/birdclef-2024/train_metadata.csv"
train_df = pd.read_csv(train_metadata_path)
train_df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg


In [17]:
train_df.describe()

,latitude,longitude,rating
count,24081.000000,24081.000000,24459.000000
mean,32.537040,43.640699,3.843493
std,19.440382,50.191352,1.100840
min,-43.524000,-171.765400,0.000000
25%,17.160100,2.545700,3.000000
50%,37.155100,26.687600,4.000000
75%,49.114400,85.319300,5.000000
max,71.964000,177.447800,5.000000


### Prepare dataframe pointing to bird call audio data in wav format

In [ ]:
from nightingale.data_pipeline.audio_preprocessor import AudioPreprocessor

# convert data if necessary
preprocessor = AudioPreprocessor()
# Read train meta data. Convert to yamnet compatible format. Skip if file already exists.
preprocessor.process_folder(
    input_root="../data/birdclef-2024/train_audio",
    output_root="../data/birdclef-2024/train_audio_16",
)

Processing 4 folders
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC214005.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC214006.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC19645.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC174932.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC177780.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC140254.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC191169.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC136837.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/integr/XC842970.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/integr/XC401712.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/integr/XC397702.wav
Skipped (already exists): ../data/birdclef-2024/train_audio

### Filter the Metadata for the folder that are actually existing

In [ ]:
from nightingale.data_pipeline.filter_birdclef_data import load_birdclef_metadata

# filter the metadata csv file for th data that is actually in the data folder
filtered_bird_df, num_of_bird_classes_in_dataset = load_birdclef_metadata(metadata_path="../data/birdclef-2024/train_metadata.csv", 
                                          audio_root="../data/birdclef-2024/train_audio_16")
print(f"Number of bird classes in dataset: {num_of_bird_classes_in_dataset}")

Number of bird classes in dataset: 3


### Split data: Training, Validation and Test

In [20]:
from nightingale.data_pipeline.audio_dataset_splitter import AudioDatasetSplitter

data_split = AudioDatasetSplitter()
train_ds, val_ds, test_ds = data_split.build(filtered_bird_df)

# Load the Yamnet Model and extract Embeddings

In [21]:
#object of the yamnet Model
yam = YamnetEmbedding()

In [22]:
train_ds = train_ds.map(yam).unbatch()
val_ds = val_ds.map(yam).unbatch()
test_ds = test_ds.map(yam).unbatch()
train_ds.element_spec

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

## Create Model for Classifier

In [23]:
bird_class_model = ClassifierHead(num_classes=num_of_bird_classes_in_dataset)

bird_class_model.summary()

Model: "classifier_head_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
bird_class_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

# Train the model

#### Train classifier

In [25]:
history = bird_class_model.fit(train_ds,
                       epochs=20,
                       validation_data=val_ds,
                       callbacks=callback)

Epoch 1/20


/workspaces/nightingale/.venv/lib/python3.11/site-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


     17/Unknown 3s 3ms/step - accuracy: 0.7222 - loss: 0.6694

/workspaces/nightingale/.venv/lib/python3.11/site-packages/keras/src/trainers/epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.8949 - loss: 0.3259 - val_accuracy: 0.9314 - val_loss: 0.1980
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9639 - loss: 0.1086 - val_accuracy: 0.9346 - val_loss: 0.2097
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9851 - loss: 0.0609 - val_accuracy: 0.9477 - val_loss: 0.1863
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9894 - loss: 0.0448 - val_accuracy: 0.9281 - val_loss: 0.1908
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9894 - loss: 0.0353 - val_accuracy: 0.9412 - val_loss: 0.1769
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9958 - loss: 0.0231 - val_accuracy: 0.9379 - val_loss: 0.1791
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9968 - loss: 0.0205 - val_accuracy: 0.9379 - val_loss: 0.2042
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9968 - loss: 0.0138 - val_accuracy: 0.9346 - val_loss: 0.2004
Ep

2025-11-24 13:16:18.535927: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.9346 - val_loss: 0.2200
Epoch 17/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 0.9346 - val_loss: 0.2270
Epoch 18/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 0.9346 - val_loss: 0.2288
Epoch 19/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.9379 - val_loss: 0.2358
Epoch 20/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.9346 - val_loss: 0.2310


#### Evaluate classifier

In [14]:
loss, accuracy = bird_class_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

      7/Unknown 1s 43ms/step - accuracy: 0.6563 - loss: 1.0617

2025-11-24 13:14:27.048688: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 82182144 exceeds 10% of free system memory.


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.8421 - loss: 0.6074
Loss:  0.6073823571205139
Accuracy:  0.8421052694320679


# Create the MLFlow Server

In [26]:
# import mlflow
# from mlflow import MlflowClient

# TRACKING_URI_LOCAL = "http://host.docker.internal:5757"

# client = MlflowClient(tracking_uri=TRACKING_URI_LOCAL)

import mlflow
# from mlflow import MlflowClient

# At the beginning of your Python script
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

False

In [27]:
import os
print("Env:", os.getenv("MLFLOW_TRACKING_URI"))
print("From MLflow:", mlflow.get_tracking_uri())

Env: None
From MLflow: file:///workspaces/nightingale/notebooks/mlruns


### Create experiment 
RUN THE FOLLOWING CODE BLOCK ONLY ONCE FOR INITIAL EXPERIMENT SETUP!!

In [ ]:
# experiment_description = (
#     "Nightingale is a bird call classification project."
# )

# experiment_tags = {
#     "project_name": "nightingale",
#     "mlflow.note.content": experiment_description,
# }

# # only run following command once to create the experiment after the server has been started for the first time
# # client.create_experiment(name="Nightingale Bird Call Classification", tags=experiment_tags)
# mlflow.set_experiment(
#     experiment_name="/Workspace/Users/ephraim.eckl@posteo.de/nightingale",
#     experiment_id="2165278269360514"
# )

In [28]:
# Assemble the metrics we're going to write into a collection
metrics = {"Loss": loss, "Accuracy": accuracy}
params = {
    "num_bird_classes": num_of_bird_classes_in_dataset,
    "optimizer": "adam",
    "loss_function": "SparseCategoricalCrossentropy",
    "loss_from_logits": True,
    "epochs": len(history.epoch),
    "batch_size": 32,
    "early_stopping_monitor": "loss",
    "early_stopping_patience": 3,
}

# Initiate the MLflow run context
with mlflow.start_run() as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Take one batch from the dataset
    x_batch, y_batch = next(iter(train_ds))

    # Convert to numpy (MLflow expects numpy or tensor-like input, not a tf.data.Dataset)
    sample_input = x_batch.numpy()
    sample_output = bird_class_model.predict(sample_input)

    # Infer signature from data
    signature = mlflow.models.infer_signature(sample_input, sample_output)

    print("Shape of input_example:", sample_input.shape)
    # Log an instance of the trained model for later use
    model_info = mlflow.keras.log_model(model=bird_class_model, name = "Bird-Call-Classifier-Head", signature=signature, pip_requirements=['keras==3.10.0'], registered_model_name="nightingale-dev.default.Reg-Bird-Call-Classifier-Head")
#     # mlflow.sklearn.log_model(sk_model=rf, input_example=X_val, name=artifact_path)
    

MissingConfigException: Yaml file '/workspaces/nightingale/notebooks/mlruns/0/meta.yaml' does not exist.

#### Run inference on a bird call audio sample (YAMNet + classifier head)

In [ ]:
# wav = load_wav_16k_mono(filtered_bird_df[filtered_bird_df['fold'] == 3]['filename'].values[1])
# scores, embeddings, spectrogram = model(wav)
# result = bird_class_model(embeddings).numpy()

# inferred_class = bird_classes[result.mean(axis=0).argmax()]
# print(f'The main sound is: {inferred_class}')

In [ ]:
# bird_class_model.save('bird_classifier_head.keras')